In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re

# Data cleaning

The goal of this code is the clean the raw data and to merge it with the API

In [ ]:
#read data with decimals as dot
df = pd.read_csv('../../data/amazon_search_hedonic.csv', sep = ";", decimal = ".")

In [ ]:
#remove duplicates and empty variables
df1 = len(df)
print(df1)
df = df.drop_duplicates(subset = ['review_title', 'review', 'date', 'name', 'profile_url'])
df2 = len(df)
print(df2)
print('Removed', df1 - df2, 'duplicates')
df = df[df['country'] == 'the United States']
df3 = len(df)
print('Removed', df3 - df2, 'non USA reviews')
print(df3)
df = df[df['review'] != '']
df4 = len(df)
print('Removed', df4 - df3, 'empty reviews')
#remove reviews with na
df = df.dropna(subset = ['review'])
df5 = len(df)
print('Removed', df5 - df4, 'na reviews')


In [ ]:
# correct mistake in encoding numerical data
df['helpful'] = df['helpful'].str.replace(',', '')

df['helpful'] = pd.to_numeric(df['helpful'])

In [ ]:
#impute 0 for missing helpful votes (these are not truly missing, the scraper just didn't pick 0 votes up)
df['helpful'] = df['helpful'].fillna(0)

In [ ]:
#add id so can reorder into original order for concatting and merging
df['id'] = range(0, len(df))

In [ ]:
#merge with category data
def remove_last_digit(string):
    return re.sub(r'\d+$', '', string)

df['asin_url_clean'] = df['asin_url'].apply(remove_last_digit)
links_data = pd.read_csv(f"../../gen/input/amazon_links_per_category.csv", delimiter = ";")
#remove duplicates for links_data product_link
links_data = links_data.drop_duplicates(subset = ['product_link'])
df = pd.merge(df, links_data, left_on = 'asin_url_clean', right_on = 'product_link', how = 'left')

In [ ]:
#drop columns
df = df.drop(columns = ['brand', 'model', 'first_spec', 'asin_url_clean', 'product_link', 'page_link', 'name_y'])

In [ ]:
#summary
print(df['category'].value_counts())

# Add variables

In [ ]:
import ast
def count_photos(pics):
    if pics == 'no':
        return 0
    else:
        return len(ast.literal_eval(pics))

df['n_pictures'] = df['picture'].apply(count_photos)

In [ ]:
#length of review
df['review_len'] = df['review'].str.len()

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%B %d, %Y')

In [ ]:
df['days_diff'] = (pd.to_datetime(df['scrape_date']) - df['date']).dt.days

In [ ]:
#print the number of observations per category, and the number of unique products, and the number of non=nan profile values
df_descriptives = df.groupby('category').agg({'category': 'size',
                                  'asin': 'nunique',
                                  'profile': lambda x: sum(~x.isna()),
                                  'picture': lambda x: sum(x != 'no'),
                                  'n_pictures': 'mean',
                                  'review_len' : 'mean',
                                  'helpful' : 'mean',
                                    'rating' : 'mean',
                                    #add average date
                                    'days_diff' : 'mean'

                                  })
#df_descriptives.columns = ['n_obs', 'n_products', 'n_profiles', 'n_pictures', 'photos_per_review', 'helpful_mean', 'rating_mean']
df_descriptives['profile'] = df_descriptives['profile']/df_descriptives['category']*100
df_descriptives['picture'] = df_descriptives['picture']/df_descriptives['category']*100

#change order of category
df_descriptives = df_descriptives.reindex(df['category'].unique())
#add average at bottom
df_descriptives.loc['average'] = df_descriptives.mean()
#sum at the bottom, except for the average row
df_descriptives.loc['sum'] = df_descriptives.iloc[:-1, :].sum()
#round values to 3 decimals
df_descriptives = df_descriptives.round(2)
#to percentage
df_descriptives['profile'] = df_descriptives['profile'].astype(str) + '%'
df_descriptives['picture'] = df_descriptives['picture'].astype(str) + '%'
print(df_descriptives)
#save to excel
df_descriptives.to_excel('../../gen/output/amazon_descriptives.xlsx')

In [ ]:
#proportion of reviews with at least 1 helpful vote
len(df[df['helpful'] > 0]) / len(df)

# review cleaning

In [ ]:
#CHECK NLTK VERSION
import nltk
nltk.__version__

In [ ]:
#remove punctuation from review
print(df['review'][0])
df['review_clean'] = df['review'].str.replace('[^\w\s]','')
print(df['review_clean'][0])
#lowercase everything
df['review_clean'] = df['review_clean'].str.lower()
print(df['review_clean'][0])
#remove stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['review_clean'] = df['review_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(df['review_clean'][0])
#stemming
from nltk.stem import PorterStemmer
porter = PorterStemmer()
df['review_clean'] = df['review_clean'].apply(lambda x: ' '.join([porter.stem(word) for word in x.split()]))
print(df['review_clean'][0])
#tokenize review
#from nltk.tokenize import word_tokenize
#df['review_clean'] = df['review_clean'].apply(word_tokenize)
#print(df['review_clean'][0])


In [ ]:
hedonic = 3*["Hedonic"] + 3*['Utilitarian'] + 3*["Hedonic"] + 3*['Utilitarian']
search = 6*["Search"] + 6*["Experience"]

In [ ]:
categories = pd.DataFrame({'category': df['category'].unique(), 'hedonic_utilitarian': hedonic, 'search_experience': search})
categories

In [ ]:
#merge with data
df = pd.merge(df, categories, on = 'category', how = 'left')

In [ ]:
#Replace underscore with space and capitalize every word in category
df['category'] = df['category'].str.replace('_', ' ').str.title()

In [ ]:
#save to csv
df.to_csv('../../gen/input/amazon_search_hedonic_clean.csv', sep = ";", index = False)

# merge with api data

In [ ]:
df = pd.read_csv('../../gen/input/amazon_search_hedonic_clean.csv', sep = ";")

In [ ]:
def get_word_count(review):
    return len(review.split(' '))

df['wordcount'] = df['review'].apply(get_word_count)

In [ ]:
api_data = pd.read_csv('../../gen/input/api_info.csv', sep = ";")
api_data

In [ ]:
#make empty dataframe with same column names
api_data_clean = pd.DataFrame(columns = api_data.columns)

#loop over asins
for asin in df['asin'].unique():
    #get the data for the asin
    asin_data = api_data[api_data['asin'] == asin]
    #set datetime index
    asin_data.index = pd.to_datetime(asin_data['date'])
    asin_data = asin_data.interpolate(method='time')
    asin_data = asin_data.fillna(method='bfill')
    asin_data = asin_data.fillna(method='ffill')
    #add to the dataframe
    api_data_clean = api_data_clean.append(asin_data)
    

In [ ]:
#save api clean
api_data_clean.to_csv('../../gen/input/api_info_clean.csv', sep = ";")

In [ ]:
#open api clean
api_data_clean = pd.read_csv('../../gen/input/api_info_clean.csv', sep = ";", index_col = 0)

In [ ]:
def return_features(asin):
    df_agg = api_data_clean[api_data_clean['asin'] == asin][['price_new', 'rating']].agg(['mean', 'min', 'max', 'median'])
    reshaped_df = pd.DataFrame(df_agg.values.reshape(1, -1))
    reshaped_df.columns = [f'{col}_{idx}' for idx in df_agg.index for col in df_agg.columns]
    reshaped_df['asin'] = asin
    return reshaped_df

In [ ]:
api_data_clean[api_data_clean['asin'] == "fdsaf"][['price_new', 'rating']].agg(['mean', 'min', 'max', 'median'])

In [ ]:
asin_info = pd.DataFrame(columns = ['price_new_mean', 'rating_mean', 'price_new_min', 'rating_min',
       'price_new_max', 'rating_max', 'price_new_median', 'rating_median',
       'asin'])
for asin in df['asin'].unique():
    asin_info = pd.concat([asin_info, return_features(asin)], axis = 0)

asin_info

In [ ]:
asin_info.reset_index(drop = True, inplace = True)

In [ ]:
#merge asin_info
df = pd.merge(df, asin_info, on = 'asin', how = 'left')

In [ ]:
#date to datetime
api_data_clean['date'] = pd.to_datetime(api_data_clean['date'])

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
api_data_clean.rename({'price_new' : 'price_rd', 'rating' : 'rating_rd', 'reviews' : 'reviews_rd'}, axis = 1, inplace = True)

In [ ]:
#merge df with api_data_clean on asin and nearest date
df_m = pd.merge_asof(df.sort_values('date'), api_data_clean.sort_values('date'), on = 'date', by = 'asin', direction = 'nearest')

In [ ]:
df_m['scrape_date'] = pd.to_datetime(df_m['scrape_date'])

In [ ]:
api_data_clean_sd = api_data_clean.drop(columns = ['price_used', 'oos_new', 'oos_used', 'listed_since'])

In [ ]:
api_data_clean_sd.rename({'price_rd' : 'price_sd', 'rating_rd' : 'rating_sd', 'reviews_rd' : 'reviews_sd', 'date' : 'scrape_date'}, axis = 1, inplace = True)

In [ ]:
df_m = pd.merge_asof(df_m.sort_values('scrape_date'), api_data_clean_sd.sort_values('scrape_date'), on = 'scrape_date', by = 'asin', direction = 'nearest')

In [ ]:
#order df_m based on id
df_m = df_m.sort_values('id')

In [ ]:
df_m['price_diff'] = df_m['price_sd'] - df_m['price_rd']
df_m['rating_diff'] = df_m['rating_sd'] - df_m['rating_rd']
df_m['reviews_diff'] = df_m['reviews_sd'] - df_m['reviews_rd']

In [ ]:
#fill in reviews na using mean on review date
df_m['price_rd'] = df_m['price_rd'].fillna(df_m['price_rd'].mean())
df_m['rating_rd'] = df_m['rating_rd'].fillna(df_m['rating_rd'].mean())
df_m['reviews_rd'] = df_m['reviews_rd'].fillna(df_m['reviews_rd'].mean())

#varaibles at scrape date
df_m['price_sd'] = df_m['price_sd'].fillna(df_m['price_sd'].mean())
df_m['rating_sd'] = df_m['rating_sd'].fillna(df_m['rating_sd'].mean())
df_m['reviews_sd'] = df_m['reviews_sd'].fillna(df_m['reviews_sd'].mean())

#difference
df_m['price_diff'] = df_m['price_diff'].fillna(df_m['price_diff'].mean())
df_m['rating_diff'] = df_m['rating_diff'].fillna(df_m['rating_diff'].mean())
df_m['reviews_diff'] = df_m['reviews_diff'].fillna(df_m['reviews_diff'].mean())

#fill in for price and rating
df_m['price_new_mean'] = df_m['price_new_mean'].fillna(df_m['price_new_mean'].mean())
df_m['rating_mean'] = df_m['rating_mean'].fillna(df_m['rating_mean'].mean())
df_m['price_new_min'] = df_m['price_new_min'].fillna(df_m['price_new_min'].mean())
df_m['rating_min'] = df_m['rating_min'].fillna(df_m['rating_min'].mean())
df_m['price_new_max'] = df_m['price_new_max'].fillna(df_m['price_new_max'].mean())
df_m['rating_max'] = df_m['rating_max'].fillna(df_m['rating_max'].mean())
df_m['price_new_median'] = df_m['price_new_median'].fillna(df_m['price_new_median'].mean())
df_m['rating_median'] = df_m['rating_median'].fillna(df_m['rating_median'].mean())


In [ ]:
#rename columns
df_m = df_m.rename(columns = {'name_x' : 'name'})

In [ ]:
#save csv
df_m.to_csv('../../gen/output/amazon_search_hedonic_clean_merged.csv', sep = ";", index = False)

In [ ]:
#open df_m
df_m = pd.read_csv('../../gen/output/amazon_search_hedonic_clean_merged.csv', sep = ";")

# Select helpful versus unhelpful reviews

In [ ]:
#create column that indicates if a review as 0 helpful votes, or more than 0
df_m['helpful_cat'] = 0
df_m.loc[df_m['helpful'] > 0, 'helpful_cat'] = 1

In [ ]:
sample_size = df_m[df_m['helpful_cat'] == 1].shape[0]
df_m_0 = df_m[df_m['helpful_cat'] == 0].sample(sample_size, random_state = 42, replace = False)
df_m_0_1 = pd.concat([df_m_0, df_m[df_m['helpful_cat'] == 1]])

In [ ]:
#sort index
df_m_0_1 = df_m_0_1.sort_index()
df_m_0_1

In [ ]:
#reset index
df_m_0_1.reset_index(drop = True, inplace = True)

In [ ]:
#save as csv
df_m_0_1.to_csv('../../gen/output/balanced_dataset.csv', sep = ";", index = False)

In [ ]:
df_m_0_1[['review', 'helpful']].to_csv('../../gen/output/balanced_review_helpful.csv', sep = ";", index = False)

# normalize and select training variables

In [ ]:
df_m = pd.read_csv('../../gen/output/balanced_dataset.csv', sep = ";")

In [ ]:
#get a frequency table for reviews_diff with buckets of 10000
pd.cut(df_m['reviews_diff'], bins = np.arange(-10000, 10000, 1000)).value_counts()


In [ ]:
#reset index df_m
df_m = df_m.reset_index(drop = True)

In [ ]:
df_m['rating_deviation_sd'] = df_m['rating'] - df_m['rating_sd']

df_m['rating_deviation_rd'] = df_m['rating'] - df_m['rating_rd']

In [ ]:
df_selection = df_m[['helpful', 'review', 'days_diff', 'rating', 'hedonic_utilitarian', 'search_experience', 'video',
                     'price_sd', 'price_diff', 'rating_diff', 'rating_deviation_sd', 'reviews_sd', 'reviews_diff',
                     'n_pictures', 'review_len',
                     'price_new_mean', 'rating_mean', 'price_new_min', 'rating_min', 'price_new_max', 'rating_max', 'price_new_median', 'rating_median',
                     'category']]

In [ ]:
df_selection['hedonic'] = df_selection['hedonic_utilitarian'].map({'Hedonic' : 1, 'Utilitarian' : 0})
df_selection['experience'] = df_selection['search_experience'].map({'Experience' : 1, 'Search' : 0})
df_selection['has_video'] = np.where(df_selection['video'].notna(), 1, 0)

In [ ]:
#make dummies from the category variable
dummies = pd.get_dummies(df_selection['category'], drop_first=True)
df_selection= pd.concat([df_selection, dummies], axis=1)


In [ ]:
df_selection = df_selection.drop(columns = ['hedonic_utilitarian',
       'search_experience', 'video'])

In [ ]:
df_selection = df_selection.drop(columns = ['category'])

In [ ]:
df_selection.to_csv('../../gen/output/balanced_selection_variables.csv', sep = ";", index = False)

In [ ]:
df_selection[['helpful', 'review']]

In [ ]:
#concat df_normalized with reviews and helpful
df_normalized = pd.concat([df_normalized, df_selection[['helpful', 'review']]], axis = 1)

In [ ]:
df_normalized = pd.concat([df_normalized, df_selection[['hedonic', 'experience', 'has_video']]], axis = 1)

In [ ]:
#make correlation matrix of df_normalized
df_normalized.corr()

In [ ]:
#save to csv
df_normalized.to_csv('../../gen/output/normalized_selection.csv', sep = ";", index = False)